In [1]:
import os
from functools import wraps
from collections import defaultdict
from tqdm import tqdm

import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
import copy
import random
import time
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, utils, datasets
from argparse import ArgumentParser
from torchvision import transforms as tt
from PIL import Image
from utils import AverageMeter

/home/haken/.conda/envs/splitfedssl/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils.utils import *
# from utils.training import *
# from utils.training_batch import *
from utils.model import *
from utils.BYOL_models import *

In [3]:
# set manual seed for reproducibility
seed = 1234

In [4]:
# general reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [5]:
# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
# ('mnist', 'femnist', 'fmnist', 'cifar10', 'cifar100', 'svhn')
data_path = "./data"
dataset = "cifar10"
# ('noniid-labeldir', 'iid', 'default') default only for femnist
partition = "iid"
test_batch = 128

In [7]:
# Hyperparameters_List (H) = [rounds, client_fraction, number_of_clients, number_of_training_rounds_local, local_batch_size, lr_client]

stop_gradient = True
has_predictor = True
OneLayer = "1_layer"
TwoLayer = "2_layer"
predictor_network=TwoLayer
global_epochs = 1000
# global_epochs = 3
client_fraction = 1.0
client_num = 5
local_epoch = 5
batch_size = 32
lr = 3e-4
partition = 'iid'
norm = 'bn'
alpha_partition = 0.5
sch_flag = False
iid = False
# 8 batch_size avg_freq = 40(25)
# 16 batch_size avg_freq = 20(25)
# 32 batch_size avg_freq = 2(125)
# 32 batch_size avg_freq = 5(50)
# 32 batch_size avg_freq = 10(25)
# 32 batch_size avg_freq = 25(10)
# 32 batch_size avg_freq = 50(5)
# 64 batch_size avg_freq = 5(25)
# 128 batch_size avg_freq = 3(21)
avg_freq = 10


data_portion = 1.0
noniid_ratio = 1.0
# noniid_ratio = 0.55

# save_path = f"./model/SplitFSSLMaxpool_resnet18/resnet18Maxpooling_cifar10_{batch_size}_{noniid_ratio}_{client_num}"
save_path = f"./model/SplitFSSL_BYOL_Avg25times/resnet18Maxpooling_cifar10_{batch_size}_{avg_freq}_{partition}_{client_num}"
# save_path = f"./model/SplitFSSL_BYOL32_DifAvgtimes/resnet18Maxpooling_cifar10_{batch_size}_{avg_freq}_{noniid_ratio}_{client_num}"
H = [global_epochs, client_fraction, client_num, local_epoch, batch_size, lr]

In [8]:
def save_checkpoint(state, checkpoint, filename= 'checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    os.makedirs(checkpoint, exist_ok=True)
    torch.save(state, filepath)
    print(f'global epoch {state["glepoch"]} saved')

In [9]:

# partition
net_dataidx_map, net_dataidx_map_test, traindata_cls_counts, testdata_cls_counts = partition_data(dataset, data_path, partition, client_num)

# get dataloader
train_loader_list = []
test_loader_list = []
for idx in range(client_num):
    
    dataidxs = net_dataidx_map[idx]
    if net_dataidx_map_test is None:
        dataidx_test = None 
    else:
        dataidxs_test = net_dataidx_map_test[idx]

    train_dl_local, test_dl_local, train_ds_local, test_ds_local = get_dataloader(dataset, 
                                                                   data_path, batch_size, test_batch, 
                                                                   dataidxs, dataidxs_test)
    train_loader_list.append(train_dl_local)
    test_loader_list.append(test_dl_local)

Files already downloaded and verified
Files already downloaded and verified
partition: iid
Data statistics Train: {0: {0: 987, 1: 1003, 2: 982, 3: 1057, 4: 966, 5: 950, 6: 1009, 7: 1015, 8: 1027, 9: 1004}, 1: {0: 1002, 1: 999, 2: 982, 3: 1020, 4: 1035, 5: 992, 6: 1067, 7: 923, 8: 997, 9: 983}, 2: {0: 1009, 1: 1012, 2: 1037, 3: 974, 4: 998, 5: 1004, 6: 960, 7: 997, 8: 1003, 9: 1006}, 3: {0: 1030, 1: 938, 2: 978, 3: 993, 4: 1050, 5: 1032, 6: 969, 7: 1021, 8: 964, 9: 1025}, 4: {0: 972, 1: 1048, 2: 1021, 3: 956, 4: 951, 5: 1022, 6: 995, 7: 1044, 8: 1009, 9: 982}}
Data statistics Test:
 {0: {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 1: {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 2: {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 3: {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 4: {0: 1000, 1: 1000, 2: 1000,

In [10]:
class BasicBlock(nn.Module):
    # feature expansion
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    """ResNet
    Note two main differences from official pytorch version:
    1. conv1 kernel size: pytorch version uses kernel_size=7
    2. average pooling: pytorch version uses AdaptiveAvgPool
    """

    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.feature_dim = 512 * block.expansion

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        # after conv1 do max pooling
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.layer1 = self._make_layer(block, 64 , num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128 , num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256 , num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512 , num_blocks[3], stride=2)
        self.avgpool = nn.AvgPool2d((4, 4))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

In [11]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

In [12]:
# augmentation utils
class RandomApply(nn.Module):
    def __init__(self, fn, p):
        super().__init__()
        self.fn = fn
        self.p = p

    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)

In [13]:
class EMA:
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

In [14]:
def update_moving_average(ema_updater, ma_model, current_model):
    for current_params, ma_params in zip(
            current_model.parameters(), ma_model.parameters()
    ):
        old_weight, up_weight = ma_params.data, current_params.data
        ma_params.data = ema_updater.update_average(old_weight, up_weight)


def byol_loss_fn(x, y):
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1)

In [15]:
class MLP(nn.Module):
    def __init__(self, dim, projection_size, hidden_size=4096, num_layer=TwoLayer):
        super().__init__()
        self.in_features = dim
        if num_layer == OneLayer:
            self.net = nn.Sequential(
                nn.Linear(dim, projection_size),
            )
        elif num_layer == TwoLayer:
            self.net = nn.Sequential(
                nn.Linear(dim, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(inplace=True),
                nn.Linear(hidden_size, projection_size),
            )
        else:
            raise NotImplementedError(f"Not defined MLP: {num_layer}")

    def forward(self, x):
        return self.net(x)

In [16]:
#client model
class BYOL_Client(nn.Module):
    def __init__(
        self,
        net=ResNet18(),
        image_size=32,
        projection_size=2048,
        projection_hidden_size=4096,
        moving_average_decay=0.99,
        stop_gradient=True,
        has_predictor=True,
        predictor_network=TwoLayer,
    ):
        super().__init__()

        self.online_encoder = net
        if not hasattr(net, 'feature_dim'):
            feature_dim = list(net.children())[-1].in_features
        else:
            feature_dim = net.feature_dim
        self.online_encoder.fc = MLP(feature_dim, projection_size, projection_hidden_size)  # projector

        self.target_encoder = None
        self.target_ema_updater = EMA(moving_average_decay)

        self.stop_gradient = stop_gradient
        self.has_predictor = has_predictor
        
        # debug purpose
        # self.forward(torch.randn(2, 3, image_size, image_size), torch.randn(2, 3, image_size, image_size))
        # self.reset_moving_average()
        
    def _get_target_encoder(self):
        target_encoder = copy.deepcopy(self.online_encoder)
        return target_encoder

    def reset_moving_average(self):
        del self.target_encoder
        self.target_encoder = None

    def update_moving_average(self):
        assert (
                self.target_encoder is not None
        ), "target encoder has not been created yet"
        update_moving_average(self.target_ema_updater, self.target_encoder, self.online_encoder)

    def forward(self, image_one, image_two):
        online_proj_one = self.online_encoder(image_one)
        online_proj_two = self.online_encoder(image_two)

        # online_pred_one = self.online_predictor(online_proj_one)
        # online_pred_two = self.online_predictor(online_proj_two)

        if self.stop_gradient:
            with torch.no_grad():
                if self.target_encoder is None:
                    self.target_encoder = self._get_target_encoder()
                target_proj_one = self.target_encoder(image_one)
                target_proj_two = self.target_encoder(image_two)

                target_proj_one = target_proj_one.detach()
                target_proj_two = target_proj_two.detach()


        # loss_one = loss_fn(online_pred_one, target_proj_two.detach())
        # loss_two = loss_fn(online_pred_two, target_proj_one.detach())

        # loss = loss_one + loss_two
        return online_proj_one, online_proj_two, target_proj_one, target_proj_two

In [17]:
# server model
class BYOL_Server(nn.Module):
    def __init__(
        self,
        projection_size=2048,
        projection_hidden_size=4096,
        moving_average_decay=0.99,
        predictor_network=TwoLayer,
    ):
        super().__init__()

        self.online_predictor = MLP(projection_size, projection_size, projection_hidden_size, predictor_network)

    def forward(self, online_proj_one, online_proj_two, target_proj_one, target_proj_two):

        online_pred_one = self.online_predictor(online_proj_one)
        online_pred_two = self.online_predictor(online_proj_two)


        loss_one = byol_loss_fn(online_pred_one, target_proj_two)
        loss_two = byol_loss_fn(online_pred_two, target_proj_one)
        loss = loss_one + loss_two
        
        return loss.mean()

In [18]:
class TransformsSimCLR:
    """
    A stochastic data augmentation module that transforms any given data example randomly 
    resulting in two correlated views of the same example,
    denoted x ̃i and x ̃j, which we consider as a positive pair.
    """

    def __init__(self, size):
        s = 1
        color_jitter = torchvision.transforms.ColorJitter(
            0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
        )
        CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
        CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
        self.train_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomResizedCrop(size=size),
                torchvision.transforms.RandomHorizontalFlip(),  # with 0.5 probability
                torchvision.transforms.RandomApply([color_jitter], p=0.8),
                torchvision.transforms.RandomGrayscale(p=0.2),
                torchvision.transforms.ToTensor(),
            ]
        )

        self.test_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(size=size),
                torchvision.transforms.ToTensor(),
                # torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                # torchvision.transforms.Normalize(CIFAR100_TRAIN_MEAN, CIFAR100_TRAIN_STD)
            ]
        )

    def __call__(self, x):
        return self.train_transform(x), self.train_transform(x)


In [19]:
net = ResNet18()
client_model = BYOL_Client(net=net, stop_gradient=stop_gradient, has_predictor=has_predictor, predictor_network=predictor_network)
server_model = BYOL_Server()
server_model.cuda()

client_weights = [1/5 for i in range(client_num)]
client_models = [copy.deepcopy(client_model).cuda() for idx in range(client_num)]
# server_models = [copy.deepcopy(server_model).cuda() for idx in range(client_num)]

optimizer_server = torch.optim.Adam(server_model.parameters(), lr = H[5]) 
optimizer_clients = [torch.optim.Adam(client_models[i].parameters(), lr = H[5]) for i in range(len(client_models))]

In [20]:
# if using checkpoint to train
epoch = 0
# checkpath = save_path + "/checkpoint.pth.tar" 
# checkpoint = torch.load(checkpath)
# epoch = checkpoint['glepoch']
# print(epoch)
# optimizer_server.load_state_dict(checkpoint['optimizer'][0])
# for localmodel in client_models:
#     localmodel.online_encoder.load_state_dict(checkpoint['state_dict'])
# for clientidx in range(client_num):
#     optimizer_clients[clientidx].load_state_dict(checkpoint['optimizer'][clientidx+1])

In [21]:
client_model

BYOL_Client(
  (online_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [22]:
class create_iterator():
    def __init__(self, iterator) -> None:
        self.iterator = iterator

    def __next__(self):
        return next(self.iterator)

In [23]:
client_iterator_list = []

for client_id in range(client_num):
    client_iterator_list.append(create_iterator(iter((train_loader_list[client_id]))))

In [24]:
def train_server(online_proj_one, online_proj_two, target_proj_one, target_proj_two, server_model):
    
    # print("shape", online_proj_one.shape)
    # print("online_proj_one", online_proj_one)
    
    
    online_proj_one.requires_grad = True
    online_proj_two.requires_grad = True
    
    online_proj_one.retain_grad()
    online_proj_two.retain_grad()

    server_model.train()
    
    # forward prop
    loss = server_model(online_proj_one, online_proj_two, target_proj_one, target_proj_two)
    
    if online_proj_one.grad is not None:
        online_proj_one.grad.zero_()
        
    if online_proj_two.grad is not None:
        online_proj_two.grad.zero_()
            
    # backward prop
    loss.backward()
    online_proj_one_grad, online_proj_two_grad = online_proj_one.grad.detach().clone(), online_proj_two.grad.detach().clone()
    # print("online_proj_one_grad", online_proj_one_grad.shape)
    # print("online_proj_two_grad", online_proj_two_grad.shape)
    
    return online_proj_one_grad, online_proj_two_grad, loss

In [25]:
def optimizer_zero_grads(optimizer_server, optimizer_clients):  # This needs to be called
    optimizer_server.zero_grad()
    for i in range(client_num):
        optimizer_clients[i].zero_grad()

In [26]:
# read one batch of image pair
def next_data_batch(client_id):
    try:
        img1, img2 = next(client_iterator_list[client_id])
        if img1.size(0) != batch_size:
            try:
                next(client_iterator_list[client_id])
            except StopIteration:
                pass
            client_iterator_list[client_id] = create_iterator(iter((train_loader_list[client_id])))
            img1, img2 = next(client_iterator_list[client_id])
    except StopIteration:
        client_iterator_list[client_id] = create_iterator(iter((train_loader_list[client_id])))
        img1, img2 = next(client_iterator_list[client_id])
    return img1, img2

In [27]:
def training(client_models, server_model, optimizer_server, optimizer_clients, rounds, batch_size, lr, C, K, local_epochs, plt_title, plt_color, cifar_data_test=None,
             test_batch_size=None, criterion=None, num_classes=None, classes_test=None, sch_flag=None):
   
    # training loss
    train_loss = []
    test_loss = []
    test_accuracy = []
    best_accuracy = 0
    avg_times = 0
    # measure time
    start = time.time()
    
    
    num_batch = len(train_loader_list[0])
    
    # writer = SummaryWriter(f'logs/SplitFSSL_BYOL32_DifAvgtimes/resnet18Maxpooling_cifar10_{batch_size}_{avg_freq}_{noniid_ratio}_{client_num}')
    writer = SummaryWriter(f'logs/SplitFSSL_BYOL_Avg25times/resnet18Maxpooling_cifar10_{batch_size}_{avg_freq}_{noniid_ratio}_{client_num}')
    global_step = 0
    for curr_round in range(epoch, rounds + 1):
        metrics = defaultdict(list)
        print(f"Global Round:", curr_round)
        w, local_loss = [], []
        
        batch_time = AverageMeter()
        data_time = AverageMeter()
        p_bar = tqdm(range(num_batch))
        
        for batch in range(num_batch):
            # print("0>", time.time() - start)
            optimizer_zero_grads(optimizer_server, optimizer_clients)
            
            online_proj_one_list = [None for _ in range(5)]
            online_proj_two_list = [None for _ in range(5)]
            target_proj_one_list = [None for _ in range(5)]
            target_proj_two_list = [None for _ in range(5)]

            # client forward
            # select 5 client to join training
            s_clients = []
            s_clients = random.sample(range(client_num), 5)
            # print("1>", time.time() - start)
            for i, client_id in enumerate(s_clients):
                # print("Client: ",i)
                # Compute a local update
                # print(i, "0>", time.time() - start)
                img1, img2 = next_data_batch(client_id)
                
                img1 = img1.cuda()
                img2 = img2.cuda()
                
                data_time.update(time.time() - start)
                # print(i, "1>", time.time() - start)
                # pass to client model
                # print("pass to client model")
                client_models[client_id].train()
                # print(i, "2>", time.time() - start)
                online_proj_one, online_proj_two, target_proj_one, target_proj_two = client_models[client_id](img1, img2)
                # print(i, "3>", time.time() - start)
                
                # store representations
                online_proj_one_list[i] = online_proj_one
                online_proj_two_list[i] = online_proj_two
                target_proj_one_list[i] = target_proj_one
                target_proj_two_list[i] = target_proj_two
                  

            # stack representations
            stack_online_proj_one = torch.cat(online_proj_one_list, dim = 0)
            stack_online_proj_two = torch.cat(online_proj_two_list, dim = 0)
            stack_target_proj_one = torch.cat(target_proj_one_list, dim = 0)
            stack_target_proj_two = torch.cat(target_proj_two_list, dim = 0)

            # print(">", time.time() - start)
            stack_online_proj_one, stack_online_proj_two, stack_target_proj_one, stack_target_proj_two = stack_online_proj_one.cuda(), stack_online_proj_two.cuda(), stack_target_proj_one.cuda(), stack_target_proj_two.cuda()
            
            # server computes
            # print("server computes")
            online_proj_one_grad, online_proj_two_grad, loss = train_server(stack_online_proj_one.detach(), stack_online_proj_two.detach(), stack_target_proj_one, stack_target_proj_two, server_model)
            local_loss.append((loss.item()))
            optimizer_server.step()
            
            # print(time.time() - start)
            # distribute gradients to clients
            # online_proj_one_grad, online_proj_two_grad = online_proj_one_grad.cpu(), online_proj_two_grad.cpu()
            gradient_dict_one = {key: [] for key in range(client_num)}
            gradient_dict_two = {key: [] for key in range(client_num)}
            
            for j in range(5):
                gradient_dict_one[j] = online_proj_one_grad[j*batch_size:(j+1)*batch_size, :]
                gradient_dict_two[j] = online_proj_two_grad[j*batch_size:(j+1)*batch_size, :]
                
            
            for i, client_id in enumerate(s_clients):
                online_proj_one_list[i].backward(gradient_dict_one[i])
                online_proj_two_list[i].backward(gradient_dict_two[i])
                optimizer_clients[client_id].step()
                client_models[client_id].update_moving_average()
            
            # if (batch+1)%10 == 0:
            #     print(f"Step [{batch}/{num_batch}]:\tLoss: {loss.item()}")
            
            del img1, img2
            writer.add_scalar("Loss/train_step", loss, global_step)
            metrics["Loss/train"].append(loss.item())
            global_step += 1
            
            batch_time.update(time.time() - start)
            start = time.time()
            #=======================================set p_bar description=======================================================
            p_bar.set_description("Train Epoch: {epoch}/{epochs:4}. Iter: {batch:4}/{iter:4}. Data: {data:.3f}s. Batch: {bt:.3f}s. Loss: {loss:.4f}.".format(
                    epoch=curr_round,
                    epochs=rounds+1,
                    batch=batch + 1,
                    iter=num_batch,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=loss.item()))
            p_bar.update()
            #=======================================set p_bar description=======================================================
            # in 32 batch size will have 250 batches, if aggregate per 10 batches will have 25 aggerations in one epoch
            # in 64 batch size will have 125 batches, if aggregate per 5 batches will have 25 aggerations in one epoch
            if batch == num_batch - 1 or ((batch+1) % avg_freq == 0):
                print("aggregate batch", batch)
                avg_times += 1
                with torch.no_grad():
                    # aggregate client models
                    for key in client_model.state_dict().keys():
                        # num_batches_tracked is a non trainable LongTensor and
                        # num_batches_tracked are the same for all clients for the given datasets
                        if "running" in key or "num_batches" in key:
                            continue
                        # elif 'target' in key:
                        #     continue
                        else:
                            temp = torch.zeros_like(client_model.state_dict()[key]).to('cuda')
                            for client_idx in s_clients:
                                temp += client_weights[client_idx] * client_models[client_idx].state_dict()[key]                        
                            client_model.state_dict()[key].data.copy_(temp)
                            for client_idx in range(len(client_models)):
                                client_models[client_idx].state_dict()[key].data.copy_(client_model.state_dict()[key])
        
        
        p_bar.close()
        # scheduler_server.step()
        for k, v in metrics.items():
            writer.add_scalar(k, np.array(v).mean(), curr_round)


        # loss
        loss_avg = sum(local_loss) / len(local_loss)
        train_loss.append(loss_avg)
        if curr_round % 5 == 0:
            optimizer_dict = []
            optimizer_dict.append(optimizer_server.state_dict())
            for client_idx in range(client_num):
                optimizer_dict.append(optimizer_clients[client_idx].state_dict())
            state_dict = client_model.online_encoder.cpu().state_dict()
            save_checkpoint({
                'glepoch': curr_round+1,
                'state_dict': state_dict,
                'optimizer': optimizer_dict,
            }, save_path)
        if curr_round % 100 == 0:
            torch.save(client_model.online_encoder.cpu().state_dict(), save_path + f"_{curr_round}_epoch.pt")
        
        
        print(f"Global round: {curr_round} | Average loss: {loss_avg}")
        # print('best_accuracy:', best_accuracy, '---Round:', curr_round, '---lr', lr, '----localEpocs--', E)

    end = time.time()
   
    print("Training Done!")
    print("Total time taken to Train: {}".format(end - start))
    print(f"Total average times : {avg_times}")

    return client_model, train_loss


In [28]:
plot_str = partition + '_' + norm + '_' + 'comm_rounds_' + str(global_epochs) + '_clientfr_' + str(
        client_fraction) + '_numclients_' + str(client_num) + '_clientepochs_' + str(
        local_epoch) + '_clientbs_' + str(batch_size) + '_clientLR_' + str(lr)
print(plot_str)

iid_bn_comm_rounds_1000_clientfr_1.0_numclients_5_clientepochs_5_clientbs_32_clientLR_0.0003


In [29]:
trained_model, train_loss = training(client_models, server_model, optimizer_server, optimizer_clients, H[0], H[4], H[5], H[1], H[2], H[3], plot_str,
                             "green")

Global Round: 0


Train Epoch: 0/1001. Iter:  313/ 313. Data: 0.147s. Batch: 0.367s. Loss: 0.0533.: 100%|█| 313/313 [01:55<00:00,  2.71it/s]


global epoch 1 saved
Global round: 0 | Average loss: 0.4955434874414255
Global Round: 1


Train Epoch: 1/1001. Iter:  313/ 313. Data: 0.148s. Batch: 0.368s. Loss: 0.0218.: 100%|█| 313/313 [01:54<00:00,  2.73it/s]


Global round: 1 | Average loss: 0.030354390051751473
Global Round: 2


...

Global round: 243 | Average loss: 0.13862319885732266
Global Round: 244


Train Epoch: 244/1001. Iter:  313/ 313. Data: 0.148s. Batch: 0.374s. Loss: 0.1524.: 100%|█| 313/313 [01:56<00:00,  2.68it/


Global round: 244 | Average loss: 0.1397564835346545
Global Round: 245


Train Epoch: 245/1001. Iter:  138/ 313. Data: 0.144s. Batch: 0.365s. Loss: 0.1674.:  44%|▍| 138/313 [00:49<00:59,  2.97it/

KeyboardInterrupt: 

In [ ]:
torch.save(client_model.online_encoder.cpu().state_dict(), save_path + "_final.pt")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss)
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()